In [ ]:
from equilibrator_api import ComponentContribution, Q_
cc = ComponentContribution()
cc.p_h = Q_(7.4)
cc.p_mg = Q_(3)
cc.ionic_strength = Q_("0.25M")
cc.temperature = Q_("293.15K")

In [ ]:
import pandas as pd
from fractions import Fraction

pathways = ["EMP", "NTS"]

def read_csv_with_fallback(file_path):
    for sep in [",", "\t", ";"]:
        for encoding in ["utf-8", "latin1", "ISO-8859-1"]:
            try:
                df = pd.read_csv(file_path, sep=sep, encoding=encoding)
                return df
            except Exception:
                pass
    raise ValueError(f"Could not read file {file_path}")

df_reactions = read_csv_with_fallback("stoichiometry_pathways.csv")

selected_pathways = pathways
df_reactions = df_reactions[df_reactions["pathway"].isin(selected_pathways)]

df_mapping = read_csv_with_fallback("input_metabolite_ranges_default.csv")
metabolite_mapping = dict(zip(df_mapping["Metabolite Acronym"], df_mapping["keggID"]))

def get_compound_id(compound_name):
    return "kegg:" + metabolite_mapping.get(compound_name, compound_name)

parsed_reactions = {}
free_energy_results = []

for index, row in df_reactions.iterrows():
    reaction_name = row["pathway"]
    coeff_fractions = {}
    
    for col in row.index:
        if col in ["pathway"]:
            continue
        value = row[col]
        if value == 0:
            continue
        frac = Fraction(str(value)).limit_denominator()
        coeff_fractions[col] = frac
    
    
    substrate_parts_parse = []
    product_parts_parse = []
    
    
    substrate_parts_display = []
    product_parts_display = []
    
    for compound, frac in coeff_fractions.items():
        abs_frac = abs(frac)
        compound_id = get_compound_id(compound)
        
        
        parse_coeff = f"{float(abs_frac.numerator / abs_frac.denominator)}"
        term_parse = f"{parse_coeff} {compound_id}"
        
        
        if abs_frac.denominator == 1:
            display_coeff = f"{abs_frac.numerator}"
        else:
            display_coeff = f"{abs_frac.numerator}/{abs_frac.denominator}"
        term_display = f"{display_coeff} {compound}"
        
        if frac < 0:
            substrate_parts_parse.append(term_parse)
            substrate_parts_display.append(term_display)
        else:
            product_parts_parse.append(term_parse)
            product_parts_display.append(term_display)
    
    reaction_string_parse = " + ".join(substrate_parts_parse) + " = " + " + ".join(product_parts_parse)
    reaction_string_display = " + ".join(substrate_parts_display) + " = " + " + ".join(product_parts_display)
    
    parsed_reaction = cc.parse_reaction_formula(reaction_string_parse)
    parsed_reactions[reaction_name] = parsed_reaction
    
    standard_dg_prime = cc.standard_dg_prime(parsed_reaction)
    
    dg_mean = round(standard_dg_prime.value.magnitude, 2)
    dg_uncertainty = round(standard_dg_prime.error.magnitude, 2)
    
    free_energy_results.append({
        "Pathway": reaction_name,
        "dG_st": dg_mean,
        "Uncertainty": dg_uncertainty
    })
    
    print(f"Parsed reaction for {reaction_name}: {reaction_string_display}")
    print(f"dG_st for {reaction_name}: {dg_mean} ± {dg_uncertainty} kJ/mol")

df_free_energy = pd.DataFrame(free_energy_results)
df_free_energy.to_csv("standard_dg_pathways.csv", index=False)
print("\nFree energy table saved")

In [ ]:
from equilibrator_api import Q_ 
import equilibrator_custom_functions_my as eq 
import numpy as np 
import math
import matplotlib.pyplot as plt 
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import ImageGrid
import custom_plot_functions as cpf

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>div.output_scroll {height: 50em; }</style>"))

plt.rc( 'axes', grid=True  )
plt.rc( 'figure', figsize = (7,4), dpi=96)
plt.rc( 'axes', linewidth=1 )
plt.rc( 'lines', linewidth=2 )

In [ ]:
#Setting physiological conditions as a Python dictionary
cellular_conditions = {
            "p_h": str(Q_("7.4")),
            "p_mg": str(Q_(3)),
            "ionic_strength": str(Q_('250 mM')),
            "temperature": str(Q_(293.15,"K")),
            "kcat_source": "fwd",
}

#Generating Compound Settings with the default eQuilibrator bounds
cs_default_bounds = eq.obtain_compound_settings("input_metabolite_ranges_default", custom_bounds = True)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
cs_default_bounds = eq.obtain_compound_settings("input_metabolite_ranges_default", custom_bounds = True)

met1 = "NADH"
met2 = "NAD"
met3 = "ATP"
met4 = "ADP"
nn = 10

conc_metabolite1 = cs_default_bounds.get_bounds(met1, bound_type='upper')
conc_metabolite2 = cs_default_bounds.get_bounds(met2, bound_type='upper')
conc_metabolite3 = cs_default_bounds.get_bounds(met3, bound_type='upper')
conc_metabolite4 = cs_default_bounds.get_bounds(met4, bound_type='upper')
total_met1_2_conc = conc_metabolite1 + conc_metabolite2
total_met3_4_conc = conc_metabolite3 + conc_metabolite4
met1_2_ratio_range = np.logspace(-3, 1, nn)
met3_4_ratio_range = np.logspace(-1, 3, nn)
met1_2_tuple = (met1, met2, met1_2_ratio_range, total_met1_2_conc)
met3_4_tuple = (met3, met4, met3_4_ratio_range, total_met3_4_conc)

test_met = 'Pi'
test_values = [0.0001, 0.030]  # in (M) 

all_mdfs_results = {}

# Loop over each test concentration.
for test_val in test_values:
    cs_default_bounds = eq.change_bounds(cs_default_bounds,
                                         [(test_met, Q_(test_val, 'M'), Q_(test_val, 'M'))])
    
    # Run MDF analysis for each selected pathway.
    mdfs_results = {}
    mdf_grids = {}
    for pathway in pathways:
        mdfs, mdf_grid = eq.MDF_double_ratio_sweep(met1_2_tuple, met3_4_tuple, pathway,
                                             cs_default_bounds, cellular_conditions,
                                             custom_dGs=True, y=0)
        mdfs_results[pathway] = mdfs
        mdf_grids[pathway] = mdf_grid 
    all_mdfs_results[test_val] = {
        'mdfs': mdfs_results,
        'mdf_grids': mdf_grids  
    }

In [ ]:
import numpy as np
import pandas as pd

# Constants
R = 0.008314  # kJ/(K*mol)
T = 293.15    # K

df_dg_st = pd.read_csv("standard_dg_pathways.csv")
pathway_dg_st = dict(zip(df_dg_st['Pathway'], df_dg_st['dG_st']))

df_stoich = pd.read_csv("stoichiometry_pathways.csv", sep=",")
stoich = {
    row["pathway"]: {
        met: coeff
        for met, coeff in row.drop("pathway").items()
        if coeff != 0
    }
    for _, row in df_stoich.iterrows()
}

df_bounds = pd.read_csv("input_metabolite_ranges_default.csv", sep=None, engine='python')

lower_bounds = dict(zip(df_bounds["Metabolite Acronym"], df_bounds["Lower Bound (M)"]))

dGr_records = []
for test_val, data in all_mdfs_results.items():
    for pw, grid in data['mdf_grids'].items():
        ν_dict = stoich[pw]
        dg0    = pathway_dg_st[pw]
        nn     = grid.shape[0]
        for i in range(nn):
            for j in range(nn):
                entry = grid[i][j]

                
                
                concs = {
                    acro: conc.magnitude
                    for acro, conc in zip(entry.compound_ids,
                                          entry.compound_df.concentration)
                }

                
                Qr = 1.0
                for met, ν in ν_dict.items():
                    if met in concs:
                        c = concs[met]
                    else:
                        
                        try:
                            c = lower_bounds[met]
                        except KeyError:
                            raise KeyError(
                                f"No MDF data *and* no lower‐bound for '{met}' in bounds CSV"
                            )
                    Qr *= c ** ν

                dGr = dg0 + R * T * np.log(Qr) if Qr > 0 else np.inf
                dGr_records.append({
        'Test Metabolite': test_met,   
        'Test Value':      test_val,   
        'Pathway':         pw,
        'i':               i,
        'j':               j,
        'dGr (kJ/mol)':    dGr,
        'Q':               Qr
    })

df_dGr = pd.DataFrame(dGr_records)
df_dGr.to_csv("dGr_results_all_combinations.csv", index=False)

In [ ]:
import numpy as np
import pandas as pd
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

pio.renderers.default = 'iframe'

df = pd.read_csv("dGr_results_all_combinations.csv", sep=",")

df = df.rename(columns={"Test Value": test_met})
test_values = sorted(df[test_met].astype(float).unique())

df['ratio1'] = df['i'].map(lambda x: met1_2_ratio_range[int(x)])
df['ratio2'] = df['j'].map(lambda x: met3_4_ratio_range[int(x)])

all_dGr_grids = {}
for Pi_val, group_Pi in df.groupby(test_met):
    all_dGr_grids[Pi_val] = {}
    for pw, grp in group_Pi.groupby('Pathway'):
        nn = len(met1_2_ratio_range)
        arr = np.full((nn, nn), np.nan)
        for _, row in grp.iterrows():
            ii = int(row['i'])
            jj = int(row['j'])
            arr[ii, jj] = row['dGr (kJ/mol)']
        all_dGr_grids[Pi_val][pw] = arr

selected_pathways = pathways
test_values = sorted(all_dGr_grids.keys())
num_test = len(test_values)
num_path = len(selected_pathways)

def format_concentration(val):
    val = float(val)
    return f"{val * 1e6:.0f} µM" if val < 0.001 else f"{val:.3f} M"

fig = make_subplots(
    rows=num_test, cols=num_path,
    subplot_titles=[
        f"{pw}<br>[{test_met}] = {format_concentration(Pi_val)}"
        for Pi_val in test_values for pw in selected_pathways
    ],
    horizontal_spacing=0.15,
    vertical_spacing=0.15,
    x_title=f"[{met1}]/[{met2}]",
    y_title=f"[{met3}]/[{met4}]"
)

colorscale = [
    [0.0, "#00441b"],
    [0.25, "#238b45"],
    [0.5, "#41ab5d"],
    [0.75, "#74c476"],
    [1.0, "#c7e9c0"]
]

global_vals = np.concatenate([
    grid.ravel()
    for Pi in test_values
    for grid in all_dGr_grids[Pi].values()
])
vmin = np.nanmin(global_vals)
vmax = np.nanmax(global_vals)

if np.isclose(vmin, vmax, rtol=1e-5):
    delta = abs(vmin) * 0.01 if vmin != 0 else 0.01
    vmin -= delta
    vmax += delta

for i, Pi in enumerate(test_values):
    grids = all_dGr_grids[Pi]

    for j, pw in enumerate(selected_pathways):
        arr = grids[pw]
        row_idx, col_idx = i + 1, j + 1
        show_cb = (i == 0 and j == 0)


        fig.add_trace(
            go.Contour(
                x=met1_2_ratio_range,
                y=met3_4_ratio_range,
                z=arr.T,
                colorscale=colorscale,
                zmin=vmin,
                zmax=vmax,
                colorbar=dict(
                    title="ΔrG (kJ/mol)",
                    ticks="outside",
                    len=0.5
                ) if show_cb else None,
                showscale=show_cb
            ),
            row=row_idx, col=col_idx
        )

        
        fig.add_trace(
            go.Contour(
                x=met1_2_ratio_range,
                y=met3_4_ratio_range,
                z=arr.T,
                contours=dict(start=0, end=0, size=1, showlines=True, coloring="none"),
                line=dict(color="red", width=3),
                showscale=False
            ),
            row=row_idx, col=col_idx
        )


def get_manual_decimal_ticks(min_value, max_value):
    ticks = np.logspace(-5, 4, num=10)
    ticks = ticks[(ticks >= min_value) & (ticks <= max_value)]
    labels = [f"10<sup>{int(np.log10(t))}</sup>" for t in ticks]
    return ticks, labels

x_ticks, x_labels = get_manual_decimal_ticks(met1_2_ratio_range[0], met1_2_ratio_range[-1])
y_ticks, y_labels = get_manual_decimal_ticks(met3_4_ratio_range[0], met3_4_ratio_range[-1])

for r in range(1, num_test + 1):
    for c in range(1, num_path + 1):
        fig.update_xaxes(
            type="log", tickmode="array",
            tickvals=x_ticks, ticktext=x_labels,
            ticks="outside", row=r, col=c
        )
        fig.update_yaxes(
            type="log", tickmode="array",
            tickvals=y_ticks, ticktext=y_labels,
            ticks="outside", row=r, col=c
        )

fig.update_layout(
    width=350 * num_path,
    height=350 * num_test,
    margin=dict(l=80, r=80, b=80, t=80),
    showlegend=False
)


for ann in fig.layout.annotations:
    if ann.text == f"[{met3}]/[{met4}]":
        ann.x -= 0.02

# Save and display
#outname = f"dGr_analysis_{test_met}.pdf"
#fig.write_image(outname, format="pdf", scale=2)
#print(f"Saved → {outname}")
fig.show()

In [ ]:
import numpy as np
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.io as pio
import pandas as pd

pio.renderers.default = 'iframe'

df_dGr = pd.read_csv("dGr_results_all_combinations.csv")

def compute_max_dGr(df_dGr):
    max_dGr = (
        df_dGr
          .groupby(['Test Value', 'Pathway'])['dGr (kJ/mol)']
          .max()
          .unstack()
          .to_dict('index')
    )
    return max_dGr

max_dGr = compute_max_dGr(df_dGr)

selected_pathways = pathways
num_test = len(test_values)
num_path = len(selected_pathways)

def format_concentration(val):
    val = float(val)
    return f"{val * 1e6:.0f} µM" if val < 0.001 else f"{val:.3f} M"

# Create subplots
titles = [
    f"{pw}<br>[{test_met}] = {format_concentration(tv)}<br> ΔᵣG = {compute_max_dGr(df_dGr)[tv].get(pw, float('nan')):.0f} kJ/mol"
    for tv in test_values for pw in selected_pathways
]


fig = make_subplots(
    rows=num_test, cols=num_path,
    row_heights=[0.3] * num_test,     
    vertical_spacing=0.2,             
    horizontal_spacing=0.15,
    subplot_titles=titles,
    x_title=f"[{met1}]/[{met2}]",
    y_title=f"[{met3}]/[{met4}]"
)

# Uniform colorscale
colorscale = [[0.0, "red"], [0.5, "white"], [1.0, "green"]]

def get_manual_decimal_ticks(min_value, max_value):
    ticks = np.logspace(-5, 4, num=10)
    ticks = ticks[(ticks >= min_value) & (ticks <= max_value)]
    labels = [f"10<sup>{int(np.log10(t))}</sup>" for t in ticks]
    return ticks, labels

for i, test_val in enumerate(test_values):
    mdfs_results = all_mdfs_results[test_val]['mdfs']
    all_vals = [v for arr in mdfs_results.values() for v in np.ravel(arr)]
    limit = math.ceil(max(abs(min(all_vals, default=0)), abs(max(all_vals, default=0))) or 1)

    for j, pathway in enumerate(selected_pathways):
        row_idx, col_idx = i+1, j+1
        if pathway not in mdfs_results:
            continue
        mdf_array = mdfs_results[pathway]

        show_cb = (i == 0 and j == 0)
        contour_kwargs = dict(
            x=met1_2_ratio_range,
            y=met3_4_ratio_range,
            z=mdf_array.T,
            colorscale=colorscale,
            contours=dict(start=-limit, end=limit, size=1),
            showscale=show_cb
        )
        if show_cb:
            contour_kwargs['colorbar'] = dict(
                title="MDF (kJ/mol)",
                tickvals=[-limit, 0, limit],
                ticktext=[f"{-limit}", "0", f"{limit}"],
                ticks="outside",
                len=0.5
            )
        
        fig.add_trace(go.Contour(**contour_kwargs), row=row_idx, col=col_idx)
        
        fig.add_trace(
            go.Contour(
                x=met1_2_ratio_range, y=met3_4_ratio_range, z=mdf_array.T,
                contours=dict(start=0, end=0, size=1, showlines=True, coloring="none"),
                line=dict(color="red", width=3), showscale=False
            ), row=row_idx, col=col_idx
        )

        
x_ticks, x_labels = get_manual_decimal_ticks(met1_2_ratio_range[0], met1_2_ratio_range[-1])
y_ticks, y_labels = get_manual_decimal_ticks(met3_4_ratio_range[0], met3_4_ratio_range[-1])
for r in range(1, num_test+1):
    for c in range(1, num_path+1):
        fig.update_xaxes(type="log", tickmode="array",
                        tickvals=x_ticks, ticktext=x_labels,
                        ticks="outside", row=r, col=c)
        fig.update_yaxes(type="log", tickmode="array",
                        tickvals=y_ticks, ticktext=y_labels,
                        ticks="outside", row=r, col=c)


fig.update_layout(
    margin=dict(l=80, r=80, b=80, t=80), 
    width=350 * num_path,
    height=350 * num_test,
    showlegend=False
)


y_label = f"[{met3}]/[{met4}]"
for ann in fig.layout.annotations:
    if ann.text == y_label:
        ann.x -= 0.02

# Save and show
PDF_file_name = f'feasibility_analysis_{test_met}.pdf'
fig.write_image(PDF_file_name, format="pdf", scale=2)
fig.show()


In [ ]:
import fitz  # PyMuPDF
from PIL import Image

def pdf_to_tiff(pdf_path, tiff_path, dpi=200):
    
    pdf_document = fitz.open(pdf_path)
    
    
    images = []
    
    
    for page_num in range(len(pdf_document)):
        
        page = pdf_document.load_page(page_num)
        
        
        pix = page.get_pixmap(matrix=fitz.Matrix(dpi/72, dpi/72))
        
        
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        
        
        images.append(img)
    
    
    if images:
        images[0].save(tiff_path, save_all=True, append_images=images[1:], compression="tiff_deflate")
        print(f"TIFF file saved successfully at {tiff_path}")
    else:
        print("No images were generated.")


pdf_path1 = PDF_file_name  
tiff_file_name1 = 'feasibility_analysis' + test_met + '.tiff'
tiff_path1 = tiff_file_name1

pdf_to_tiff(pdf_path1, tiff_path1)

In [ ]:
cs_default_bounds = eq.obtain_compound_settings("input_metabolite_ranges_default", custom_bounds = True)

met1 = "NADH"
met2 = "NAD"
met3 = "ATP"
met4 = "ADP"
nn = 10

conc_metabolite1 = cs_default_bounds.get_bounds(met1, bound_type='upper')
conc_metabolite2 = cs_default_bounds.get_bounds(met2, bound_type='upper')
conc_metabolite3 = cs_default_bounds.get_bounds(met3, bound_type='upper')
conc_metabolite4 = cs_default_bounds.get_bounds(met4, bound_type='upper')
total_met1_2_conc = conc_metabolite1 + conc_metabolite2
total_met3_4_conc = conc_metabolite3 + conc_metabolite4
met1_2_ratio_range = np.logspace(-3, 1, nn)
met3_4_ratio_range = np.logspace(-1, 3, nn)
met1_2_tuple = (met1, met2, met1_2_ratio_range, total_met1_2_conc)
met3_4_tuple = (met3, met4, met3_4_ratio_range, total_met3_4_conc)

test_met = 'Ace'
test_values = [0.0002, 0.030]  # in (M) 

all_mdfs_results = {}


for test_val in test_values:
    cs_default_bounds = eq.change_bounds(cs_default_bounds,
                                         [(test_met, Q_(test_val, 'M'), Q_(test_val, 'M'))])
    

    mdfs_results = {}
    mdf_grids = {}  
    for pathway in pathways:
        mdfs, mdf_grid = eq.MDF_double_ratio_sweep(met1_2_tuple, met3_4_tuple, pathway,
                                             cs_default_bounds, cellular_conditions,
                                             custom_dGs=True, y=0)
        mdfs_results[pathway] = mdfs
        mdf_grids[pathway] = mdf_grid  
    all_mdfs_results[test_val] = {
        'mdfs': mdfs_results,
        'mdf_grids': mdf_grids  
    }

In [ ]:
import numpy as np
import pandas as pd

# Constants
R = 0.008314  # kJ/(K*mol)
T = 293.15    # K

df_dg_st = pd.read_csv("standard_dg_pathways.csv")
pathway_dg_st = dict(zip(df_dg_st['Pathway'], df_dg_st['dG_st']))

df_stoich = pd.read_csv("stoichiometry_pathways.csv", sep=",")
stoich = {
    row["pathway"]: {
        met: coeff
        for met, coeff in row.drop("pathway").items()
        if coeff != 0
    }
    for _, row in df_stoich.iterrows()
}

df_bounds = pd.read_csv("input_metabolite_ranges_default.csv", sep=None, engine='python')

lower_bounds = dict(zip(df_bounds["Metabolite Acronym"], df_bounds["Lower Bound (M)"]))

dGr_records = []
for test_val, data in all_mdfs_results.items():
    for pw, grid in data['mdf_grids'].items():
        ν_dict = stoich[pw]
        dg0    = pathway_dg_st[pw]
        nn     = grid.shape[0]
        for i in range(nn):
            for j in range(nn):
                entry = grid[i][j]

                
                concs = {
                    acro: conc.magnitude
                    for acro, conc in zip(entry.compound_ids,
                                          entry.compound_df.concentration)
                }

                
                Qr = 1.0
                for met, ν in ν_dict.items():
                    if met in concs:
                        c = concs[met]
                    else:
                        
                        try:
                            c = lower_bounds[met]
                        except KeyError:
                            raise KeyError(
                                f"No MDF data *and* no lower‐bound for '{met}' in bounds CSV"
                            )
                    Qr *= c ** ν

                dGr = dg0 + R * T * np.log(Qr) if Qr > 0 else np.inf
                dGr_records.append({
        'Test Metabolite': test_met,   
        'Test Value':      test_val,   
        'Pathway':         pw,
        'i':               i,
        'j':               j,
        'dGr (kJ/mol)':    dGr,
        'Q':               Qr
    })


df_dGr = pd.DataFrame(dGr_records)
df_dGr.to_csv("dGr_results_all_combinations.csv", index=False)


In [ ]:
import numpy as np
import pandas as pd
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

pio.renderers.default = 'iframe'

df = pd.read_csv("dGr_results_all_combinations.csv", sep=",")

df = df.rename(columns={"Test Value": test_met})
test_values = sorted(df[test_met].astype(float).unique())

df['ratio1'] = df['i'].map(lambda x: met1_2_ratio_range[int(x)])
df['ratio2'] = df['j'].map(lambda x: met3_4_ratio_range[int(x)])

all_dGr_grids = {}
for Pi_val, group_Pi in df.groupby(test_met):
    all_dGr_grids[Pi_val] = {}
    for pw, grp in group_Pi.groupby('Pathway'):
        nn = len(met1_2_ratio_range)
        arr = np.full((nn, nn), np.nan)
        for _, row in grp.iterrows():
            ii = int(row['i'])
            jj = int(row['j'])
            arr[ii, jj] = row['dGr (kJ/mol)']
        all_dGr_grids[Pi_val][pw] = arr

selected_pathways = pathways  
test_values = sorted(all_dGr_grids.keys())
num_test = len(test_values)
num_path = len(selected_pathways)

def format_concentration(val):
    val = float(val)
    return f"{val * 1e6:.0f} µM" if val < 0.001 else f"{val:.3f} M"

fig = make_subplots(
    rows=num_test, cols=num_path,
    subplot_titles=[
        f"{pw}<br>[{test_met}] = {format_concentration(Pi_val)}"
        for Pi_val in test_values for pw in selected_pathways
    ],
    horizontal_spacing=0.15,
    vertical_spacing=0.15,
    x_title=f"[{met1}]/[{met2}]",
    y_title=f"[{met3}]/[{met4}]"
)

colorscale = [
    [0.0, "#00441b"],
    [0.25, "#238b45"],
    [0.5, "#41ab5d"],
    [0.75, "#74c476"],
    [1.0, "#c7e9c0"]
]


global_vals = np.concatenate([
    grid.ravel()
    for Pi in test_values
    for grid in all_dGr_grids[Pi].values()
])
vmin = np.nanmin(global_vals)
vmax = np.nanmax(global_vals)

if np.isclose(vmin, vmax, rtol=1e-5):
    delta = abs(vmin) * 0.01 if vmin != 0 else 0.01
    vmin -= delta
    vmax += delta

for i, Pi in enumerate(test_values):
    grids = all_dGr_grids[Pi]

    for j, pw in enumerate(selected_pathways):
        arr = grids[pw]
        row_idx, col_idx = i + 1, j + 1
        show_cb = (i == 0 and j == 0)

        
        fig.add_trace(
            go.Contour(
                x=met1_2_ratio_range,
                y=met3_4_ratio_range,
                z=arr.T,
                colorscale=colorscale,
                zmin=vmin,
                zmax=vmax,
                colorbar=dict(
                    title="ΔrG (kJ/mol)",
                    ticks="outside",
                    len=0.5
                ) if show_cb else None,
                showscale=show_cb
            ),
            row=row_idx, col=col_idx
        )

        
        fig.add_trace(
            go.Contour(
                x=met1_2_ratio_range,
                y=met3_4_ratio_range,
                z=arr.T,
                contours=dict(start=0, end=0, size=1, showlines=True, coloring="none"),
                line=dict(color="red", width=3),
                showscale=False
            ),
            row=row_idx, col=col_idx
        )

def get_manual_decimal_ticks(min_value, max_value):
    ticks = np.logspace(-5, 4, num=10)
    ticks = ticks[(ticks >= min_value) & (ticks <= max_value)]
    labels = [f"10<sup>{int(np.log10(t))}</sup>" for t in ticks]
    return ticks, labels

x_ticks, x_labels = get_manual_decimal_ticks(met1_2_ratio_range[0], met1_2_ratio_range[-1])
y_ticks, y_labels = get_manual_decimal_ticks(met3_4_ratio_range[0], met3_4_ratio_range[-1])

for r in range(1, num_test + 1):
    for c in range(1, num_path + 1):
        fig.update_xaxes(
            type="log", tickmode="array",
            tickvals=x_ticks, ticktext=x_labels,
            ticks="outside", row=r, col=c
        )
        fig.update_yaxes(
            type="log", tickmode="array",
            tickvals=y_ticks, ticktext=y_labels,
            ticks="outside", row=r, col=c
        )

fig.update_layout(
    width=350 * num_path,
    height=350 * num_test,
    margin=dict(l=80, r=80, b=80, t=80),
    showlegend=False
)

for ann in fig.layout.annotations:
    if ann.text == f"[{met3}]/[{met4}]":
        ann.x -= 0.02

# Save and display
#outname = f"dGr_analysis_{test_met}.pdf"
#fig.write_image(outname, format="pdf", scale=2)
#print(f"Saved → {outname}")
fig.show()

In [ ]:
import numpy as np
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.io as pio
import pandas as pd

pio.renderers.default = 'iframe'

df_dGr = pd.read_csv("dGr_results_all_combinations.csv")

def compute_max_dGr(df_dGr):
    max_dGr = (
        df_dGr
          .groupby(['Test Value', 'Pathway'])['dGr (kJ/mol)']
          .max()
          .unstack()
          .to_dict('index')
    )
    return max_dGr

max_dGr = compute_max_dGr(df_dGr)

selected_pathways = pathways 
num_test = len(test_values)
num_path = len(selected_pathways)

def format_concentration(val):
    val = float(val)
    return f"{val * 1e6:.0f} µM" if val < 0.001 else f"{val:.3f} M"

titles = [
    f"{pw}<br>[{test_met}] = {format_concentration(tv)}<br> ΔᵣG = {compute_max_dGr(df_dGr)[tv].get(pw, float('nan')):.0f} kJ/mol"
    for tv in test_values for pw in selected_pathways
]


fig = make_subplots(
    rows=num_test, cols=num_path,
    row_heights=[0.3] * num_test,     
    vertical_spacing=0.2,            
    horizontal_spacing=0.15,
    subplot_titles=titles,
    x_title=f"[{met1}]/[{met2}]",
    y_title=f"[{met3}]/[{met4}]"
)

colorscale = [[0.0, "red"], [0.5, "white"], [1.0, "green"]]

def get_manual_decimal_ticks(min_value, max_value):
    ticks = np.logspace(-5, 4, num=10)
    ticks = ticks[(ticks >= min_value) & (ticks <= max_value)]
    labels = [f"10<sup>{int(np.log10(t))}</sup>" for t in ticks]
    return ticks, labels

for i, test_val in enumerate(test_values):
    mdfs_results = all_mdfs_results[test_val]['mdfs']
    all_vals = [v for arr in mdfs_results.values() for v in np.ravel(arr)]
    limit = math.ceil(max(abs(min(all_vals, default=0)), abs(max(all_vals, default=0))) or 1)

    for j, pathway in enumerate(selected_pathways):
        row_idx, col_idx = i+1, j+1
        if pathway not in mdfs_results:
            continue
        mdf_array = mdfs_results[pathway]

        show_cb = (i == 0 and j == 0)
        contour_kwargs = dict(
            x=met1_2_ratio_range,
            y=met3_4_ratio_range,
            z=mdf_array.T,
            colorscale=colorscale,
            contours=dict(start=-limit, end=limit, size=1),
            showscale=show_cb
        )
        if show_cb:
            contour_kwargs['colorbar'] = dict(
                title="MDF (kJ/mol)",
                tickvals=[-limit, 0, limit],
                ticktext=[f"{-limit}", "0", f"{limit}"],
                ticks="outside",
                len=0.5
            )
        
        fig.add_trace(go.Contour(**contour_kwargs), row=row_idx, col=col_idx)
        
        fig.add_trace(
            go.Contour(
                x=met1_2_ratio_range, y=met3_4_ratio_range, z=mdf_array.T,
                contours=dict(start=0, end=0, size=1, showlines=True, coloring="none"),
                line=dict(color="red", width=3), showscale=False
            ), row=row_idx, col=col_idx
        )

        
x_ticks, x_labels = get_manual_decimal_ticks(met1_2_ratio_range[0], met1_2_ratio_range[-1])
y_ticks, y_labels = get_manual_decimal_ticks(met3_4_ratio_range[0], met3_4_ratio_range[-1])
for r in range(1, num_test+1):
    for c in range(1, num_path+1):
        fig.update_xaxes(type="log", tickmode="array",
                        tickvals=x_ticks, ticktext=x_labels,
                        ticks="outside", row=r, col=c)
        fig.update_yaxes(type="log", tickmode="array",
                        tickvals=y_ticks, ticktext=y_labels,
                        ticks="outside", row=r, col=c)


fig.update_layout(
    margin=dict(l=80, r=80, b=80, t=80), 
    width=350 * num_path,
    height=350 * num_test,
    showlegend=False
)


y_label = f"[{met3}]/[{met4}]"
for ann in fig.layout.annotations:
    if ann.text == y_label:
        ann.x -= 0.02

# Save and show
PDF_file_name = f'feasibility_analysis_{test_met}.pdf'
fig.write_image(PDF_file_name, format="pdf", scale=2)
fig.show()

In [ ]:
import fitz  # PyMuPDF
from PIL import Image

def pdf_to_tiff(pdf_path, tiff_path, dpi=200):
    
    pdf_document = fitz.open(pdf_path)
    
    
    images = []
    
   
    for page_num in range(len(pdf_document)):
        
        page = pdf_document.load_page(page_num)
        
        
        pix = page.get_pixmap(matrix=fitz.Matrix(dpi/72, dpi/72))
        
        
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        
        
        images.append(img)
    
    
    if images:
        images[0].save(tiff_path, save_all=True, append_images=images[1:], compression="tiff_deflate")
        print(f"TIFF file saved successfully at {tiff_path}")
    else:
        print("No images were generated.")


pdf_path1 = PDF_file_name  
tiff_file_name1 = 'feasibility_analysis' + test_met + '.tiff'
tiff_path1 = tiff_file_name1

pdf_to_tiff(pdf_path1, tiff_path1)